<h1>Extracting and Visualizing Stock Data</h1>
<h2>Description</h2>

In [1]:
!pip install yfinance
#!pip install pandas
#!pip install requests
!pip install bs4
#!pip install plotly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 22.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 22.2 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1257 sha256=6ceb176c14fc6645584e094466289a90f930a93baafcba55b1bc5a27893412fc
  Stored in directory: /Users/akshdatiwari/Library/Caches/pip/wheels/73/2b/cb/099980278a0c9a3e57ff1a89875ec07bfa0b6fcbebb9a8cad3
Successfully built bs4


In [2]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Define Graphing Function

In [3]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

## Question 1: Use yfinance to Extract Stock Data

In [4]:
tesla = yf.Ticker('TSLA')

In [5]:
tesla_info = tesla.info

In [6]:
tesla_data = tesla.history(period='max')

In [7]:
# first five rows in tesla dataframe
tesla_data.reset_index(inplace=True)
tesla_data.head(5)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


## Question 2: Use Webscraping to Extract Tesla Revenue Data

In [8]:
url = 'https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue'

html_data = requests.get(url).text

In [9]:
soup = BeautifulSoup(html_data, 'html.parser')

In [30]:
tesla_revenue_html = pd.read_html(url)
tesla_revenue = tesla_revenue_html[1]
tesla_revenue.columns=["Date", "Revenue"]
tesla_revenue.head()

,Date,Revenue
0,2023-01-31,"$2,226"
1,2022-10-31,"$1,186"
2,2022-07-31,"$1,136"
3,2022-04-30,"$1,378"
4,2022-01-31,"$2,254"


In [31]:
tesla_revenue.head()

,Date,Revenue
0,2023-01-31,"$2,226"
1,2022-10-31,"$1,186"
2,2022-07-31,"$1,136"
3,2022-04-30,"$1,378"
4,2022-01-31,"$2,254"


In [32]:
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',|\$',"")

In [33]:
tesla_revenue.dropna(inplace=True)

tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

In [34]:
tesla_revenue.tail()

,Date,Revenue
52,2010-01-31,"$3,524"
53,2009-10-31,"$1,835"
54,2009-07-31,"$1,739"
55,2009-04-30,"$1,981"
56,2009-01-31,"$3,492"


## Question 3: Use yfinance to Extract Stock Data

In [35]:
gme = yf.Ticker('GME')

In [36]:
gme_data = gme.history(period='max')

In [37]:
gme_data.reset_index(inplace=True)
gme_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620128,1.693350,1.603296,1.691666,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712707,1.716074,1.670626,1.683250,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658002,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615921,1.662210,1.603296,1.662210,6892800,0.0,0.0


## Question 4: Use Webscraping to Extract GME Revenue Data

In [38]:
url = 'https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue'

html_data = requests.get(url).text

In [39]:
soup = BeautifulSoup(html_data, 'html.parser')

In [40]:
gme_revenue_html = pd.read_html(url)
gme_revenue = gme_revenue_html[1]
gme_revenue.columns=["Date", "Revenue"]
gme_revenue.head()

,Date,Revenue
0,2023-01-31,"$2,226"
1,2022-10-31,"$1,186"
2,2022-07-31,"$1,136"
3,2022-04-30,"$1,378"
4,2022-01-31,"$2,254"


In [41]:
# remove the comma and dollar sign from the Revenue column.
gme_revenue["Revenue"] = gme_revenue['Revenue'].str.replace(',|\$',"")

gme_revenue.dropna(inplace=True)

gme_revenue = gme_revenue[gme_revenue['Revenue'] != ""]

In [42]:
# display tail of dataframe
gme_revenue.tail()

,Date,Revenue
52,2010-01-31,"$3,524"
53,2009-10-31,"$1,835"
54,2009-07-31,"$1,739"
55,2009-04-30,"$1,981"
56,2009-01-31,"$3,492"


## Question 5: Plot Tesla Stock Graph

In [27]:
make_graph(tesla_data, tesla_revenue, 'Tesla')

/var/folders/tv/wgcfj54j1sv6jq8hv5dkxxtr0000gn/T/ipykernel_31680/2068038883.py:5: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

/var/folders/tv/wgcfj54j1sv6jq8hv5dkxxtr0000gn/T/ipykernel_31680/2068038883.py:6: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.



ValueError: could not convert string to float: '$10,389'

## Question 6: Plot GameStop Stock Graph

In [26]:
make_graph(gme_data, gme_revenue, 'GameStop')

/var/folders/tv/wgcfj54j1sv6jq8hv5dkxxtr0000gn/T/ipykernel_31680/2068038883.py:5: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

/var/folders/tv/wgcfj54j1sv6jq8hv5dkxxtr0000gn/T/ipykernel_31680/2068038883.py:6: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.



ValueError: could not convert string to float: '$1,277'

## Change Log

| Date (YYYY-MM-DD) | Version | Changed By    | Change Description        |
| ----------------- | ------- | ------------- | ------------------------- |
| 2020-11-10        | 1.1     | Malika Singla | Deleted the Optional part |
| 2020-08-27        | 1.0     | Malika Singla | Added lab to GitLab       |

<hr>

## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>

<p>
